In [1]:
import astropy.units as u
from astroduet.duet_telescope import load_telescope_parameters
from astroduet.duet_sensitivity import src_rate, bgd_sky_qe_rate, bgd_electronics, calc_exposure
from astroduet.bbmag import bb_abmag_fluence
import numpy as np
from matplotlib import pyplot as plt
from astroduet.bbmag import bb_abmag_fluence, bb_abmag

import astroduet.config as config
from astroduet.background import background_pixel_rate


%load_ext autoreload
%autoreload 2

from astropy.visualization import quantity_support
import matplotlib
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 22}

matplotlib.rc('font', **font)

In [15]:

# Band1
# 5-sigma limiting magnitude in 1, 2, 3, and 4 stacked frames.

# Account for the fact that you're co-adding the two frames here:
duet = config.Telescope()
duet.read_noise = duet.read_noise


#tot_bgd_rate = bgd_band1 + bgd_band2
bandone = duet.bandpass1
bandtwo = duet.bandpass2
exposure = 300*u.s
print()
siglimit=5
dmag = 0.05
print()
for filter_type in ['B']:
    print(filter_type)
    [bgd_band1, bgd_band2] = background_pixel_rate(duet, low_zodi=True, filter_type=filter_type, airglow_level='COS')
    tot_bgd_rate = bgd_band1

    for nframes in [1, 2, 3]:
        snr = 100
        swiftmag = 20 
        while snr > siglimit:
            swiftmag += dmag
            band1_fluence, band2_fluence = bb_abmag_fluence(duet =duet, swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, filter_type=filter_type)
            band1_rate = duet.fluence_to_rate(band1_fluence)
            band2_rate = duet.fluence_to_rate(band2_fluence)
            src_rate = band1_rate

            snr = duet.calc_snr(exposure, src_rate, tot_bgd_rate, nint=nframes)
        bbmag1, bbmag2 = bb_abmag(swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, bandone = bandone, bandtwo=bandtwo)        
        print('Band1 {} {}-σ magnitude limit: {}'.format(nframes*exposure, siglimit, bbmag1))






B
Band1 300.0 s 5-σ magnitude limit: 21.830542480741894 mag(AB)
Band1 600.0 s 5-σ magnitude limit: 22.280542480741904 mag(AB)
Band1 900.0 s 5-σ magnitude limit: 22.530542480741904 mag(AB)


In [7]:

# Band2
# 5-sigma limiting magnitude in 1, 2, 3, and 4 stacked frames.

# Account for the fact that you're co-adding the two frames here:
duet = config.Telescope()
duet.read_noise = duet.read_noise


#tot_bgd_rate = bgd_band1 + bgd_band2
bandone = duet.bandpass1
bandtwo = duet.bandpass2
exposure = 300*u.s
print()
siglimit=5
dmag = 0.05
print()
for filter_type in ['B']:
    print(filter_type)
    [bgd_band1, bgd_band2] = background_pixel_rate(duet, low_zodi=True, filter_type=filter_type)
    tot_bgd_rate = bgd_band2

    for nframes in [1, 2, 3]:
        snr = 100
        swiftmag = 20 
        while snr > siglimit:
            swiftmag += dmag
            band1_fluence, band2_fluence = bb_abmag_fluence(duet =duet, swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, filter_type=filter_type)
            band1_rate = duet.fluence_to_rate(band1_fluence)
            band2_rate = duet.fluence_to_rate(band2_fluence)
            src_rate = band2_rate

            snr = duet.calc_snr(exposure, src_rate, tot_bgd_rate, nint=nframes)
        bbmag1, bbmag2 = bb_abmag(swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, bandone = bandone, bandtwo=bandtwo)        
        print('Band2 {} {}-σ magnitude limit: {}'.format(nframes*exposure, siglimit, bbmag2))







B
Band2 300.0 s 5-σ magnitude limit: 20.84122405972026 mag(AB)
Band2 600.0 s 5-σ magnitude limit: 21.241224059720267 mag(AB)
Band2 900.0 s 5-σ magnitude limit: 21.491224059720267 mag(AB)


In [8]:

# Stacked channels
# 5-sigma limiting magnitude in 1, 2, 3, and 4 stacked frames.

# Account for the fact that you're co-adding the two frames here:
duet = config.Telescope()
duet.read_noise = (2**0.5) * duet.read_noise


#tot_bgd_rate = bgd_band1 + bgd_band2
bandone = duet.bandpass1
bandtwo = duet.bandpass2
exposure = 300*u.s
print()
siglimit=5
dmag = 0.05
print()
for filter_type in ['B']:
    print(filter_type)
    [bgd_band1, bgd_band2] = background_pixel_rate(duet, low_zodi=True, filter_type=filter_type)
    tot_bgd_rate = bgd_band1 + bgd_band2

    for nframes in [1, 2, 3]:
        snr = 100
        swiftmag = 20 
        while snr > siglimit:
            swiftmag += dmag
            band1_fluence, band2_fluence = bb_abmag_fluence(duet =duet, swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, filter_type=filter_type)
            band1_rate = duet.fluence_to_rate(band1_fluence)
            band2_rate = duet.fluence_to_rate(band2_fluence)
            src_rate = band1_rate + band2_rate

            snr = duet.calc_snr(exposure, src_rate, tot_bgd_rate, nint=nframes)
        bbmag1, bbmag2 = bb_abmag(swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, bandone = bandone, bandtwo=bandtwo)        
        print('Stacked Bands {} {}-σ magnitude limit: {}'.format(nframes*exposure, siglimit, bbmag1))







B
Stacked Bands 300.0 s 5-σ magnitude limit: 21.63054248074189 mag(AB)
Stacked Bands 600.0 s 5-σ magnitude limit: 22.030542480741897 mag(AB)
Stacked Bands 900.0 s 5-σ magnitude limit: 22.280542480741904 mag(AB)


In [ ]:
# Individual channels
# 5-sigma limiting magnitude in 1, 2, 3, and 4 stacked frames.
# Assume read noise is 3 e-
duet = config.Telescope()
duet.read_noise=3

[bgd_band1, bgd_band2] = background_pixel_rate(duet, low_zodi=True)
print('Sky background rate per pixel band1: {}'.format(bgd_band1))
print('Sky background rate per pixel band2: {}'.format(bgd_band2))

bandone = duet.bandpass1
bandtwo = duet.bandpass2
exposure = 300*u.s
print()
siglimit=5
dmag = 0.05
print()
for nframes in np.arange(1, 5):
    snr = 100
    swiftmag = 20 
    while snr > siglimit:
        swiftmag += dmag
        band1_fluence, foo = bb_abmag_fluence(duet =duet, swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K)
        band1_rate = duet.fluence_to_rate(band1_fluence)
        snr = duet.calc_snr(exposure, band1_rate, bgd_band1, nint=nframes)
    bbmag1, foo = bb_abmag(swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, bandone = bandone)
    print('Band 1 {} {}-σ magnitude limit: {}'.format(nframes*exposure, siglimit, bbmag1))
print()

for nframes in np.arange(1, 5):
    snr = 100
    swiftmag = 20 
    while snr > siglimit:
        swiftmag += dmag
        foo, band2_fluence = bb_abmag_fluence(duet =duet, swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K)
        band2_rate = duet.fluence_to_rate(band2_fluence)
        snr = duet.calc_snr(exposure, band2_rate, bgd_band2, nint=nframes)
    foo, bbmag2 = bb_abmag(swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, bandtwo = bandtwo)
    print('Band 2 {} {}-σ magnitude limit: {}'.format(nframes*exposure, siglimit, bbmag2))
print()



# Stacked channels
# 5-sigma limiting magnitude in 1, 2, 3, and 4 stacked frames.

# Account for the fact that you're co-adding the two frames here:
duet.read_noise = (2**0.5) * 3


tot_bgd_rate = bgd_band1 + bgd_band2
bandone = duet.bandpass1
bandtwo = duet.bandpass2
exposure = 300*u.s
print()
siglimit=5
dmag = 0.05
print()
for filter_type in ['C', 'B', 'A']:
    for nframes in np.arange(1, 3):
        snr = 100
        swiftmag = 20 
        while snr > siglimit:
            swiftmag += dmag
            band1_fluence, band2_fluence = bb_abmag_fluence(duet =duet, swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, filter_type=filter_type)
            band1_rate = duet.fluence_to_rate(band1_fluence)
            band2_rate = duet.fluence_to_rate(band2_fluence)
            src_rate = band1_rate + band2_rate

            snr = duet.calc_snr(exposure, src_rate, tot_bgd_rate, nint=nframes)
        bbmag1, bbmag2 = bb_abmag(swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, bandone = bandone, bandtwo=bandtwo)
        print('Stacked Bands {} {}-σ magnitude limit: {}'.format(nframes*exposure, siglimit, bbmag1))
    print()





In [ ]:
# Same as last cell, but for "reduced_baseline" configuration.

# Individual channels
# 5-sigma limiting magnitude in 1, 2, 3, and 4 stacked frames.
# Assume read noise is 3 e-
duet = config.Telescope(config='reduced_baseline')
duet.read_noise=3

[bgd_band1, bgd_band2] = background_pixel_rate(duet, low_zodi=True)
print('Sky background rate per pixel band1: {}'.format(bgd_band1))
print('Sky background rate per pixel band2: {}'.format(bgd_band2))

bandone = duet.bandpass1
bandtwo = duet.bandpass2
exposure = 300*u.s
print()
siglimit=5
dmag = 0.05
print()
for nframes in np.arange(1, 5):
    snr = 100
    swiftmag = 20 
    while snr > siglimit:
        swiftmag += dmag
        band1_fluence, foo = bb_abmag_fluence(duet =duet, swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K)
        band1_rate = duet.fluence_to_rate(band1_fluence)
        snr = duet.calc_snr(exposure, band1_rate, bgd_band1, nint=nframes)
    bbmag1, foo = bb_abmag(swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, bandone = bandone)
    print('Band 1 {} {}-σ magnitude limit: {}'.format(nframes*exposure, siglimit, bbmag1))
print()

for nframes in np.arange(1, 5):
    snr = 100
    swiftmag = 20 
    while snr > siglimit:
        swiftmag += dmag
        foo, band2_fluence = bb_abmag_fluence(duet =duet, swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K)
        band2_rate = duet.fluence_to_rate(band2_fluence)
        snr = duet.calc_snr(exposure, band2_rate, bgd_band2, nint=nframes)
    foo, bbmag2 = bb_abmag(swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, bandtwo = bandtwo)
    print('Band 2 {} {}-σ magnitude limit: {}'.format(nframes*exposure, siglimit, bbmag2))
print()



# Stacked channels
# 5-sigma limiting magnitude in 1, 2, 3, and 4 stacked frames.
# Account for the fact that you're co-adding the two frames here:
duet.read_noise = (2**0.5) * 3


tot_bgd_rate = bgd_band1 + bgd_band2
bandone = duet.bandpass1
bandtwo = duet.bandpass2
exposure = 300*u.s
print()
siglimit=5
dmag = 0.05
print()
for nframes in np.arange(1, 5):
    snr = 100
    swiftmag = 20 
    while snr > siglimit:
        swiftmag += dmag
        band1_fluence, band2_fluence = bb_abmag_fluence(duet=duet, swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K)
        band1_rate = duet.fluence_to_rate(band1_fluence)
        band2_rate = duet.fluence_to_rate(band2_fluence)
        src_rate = band1_rate + band2_rate
        
        snr = duet.calc_snr(exposure, src_rate, tot_bgd_rate, nint=nframes)
    bbmag1, bbmag2 = bb_abmag(swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, bandone = bandone, bandtwo=bandtwo)
    print('Stacked Bands {} {}-σ magnitude limit: {}'.format(nframes*exposure, siglimit, bbmag1))
print()







In [ ]:
# FOM: Time to 22 ABmag for this configuration for DUET1
# Old FOM for this 
duet = config.Telescope(config='baseline')
duet.read_noise = 3

bandone = duet.bandpass1
bandtwo = duet.bandpass2

siglimit=10
swiftmag = 21
dmag = 0.05
bbmag1 = 5*u.ABmag
bbmag_target = 22*u.ABmag
while bbmag1 < bbmag_target:
    swiftmag+=dmag
    bbmag1, bbmag2 = bb_abmag(swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, bandone = bandone, bandtwo=bandtwo)


# Low Zodi, 10-sigma case
[bgd_band1, bgd_band2] = background_pixel_rate(duet, low_zodi=True)
band1_fluence, foo = bb_abmag_fluence(duet = duet, swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K)
band1_rate = duet.fluence_to_rate(band1_fluence)
req_exp = calc_exposure(siglimit, band1_rate, bgd_band1, duet.read_noise, duet.neff)
print('Low-Zodi {}-sig, {} seconds'.format(siglimit, req_exp.value))


# High Zodi, 10-sigma case.
siglimit=10
[bgd_band1, bgd_band2] = background_pixel_rate(duet, high_zodi=True)
band1_fluence, foo = bb_abmag_fluence(duet = duet, swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K)
band1_rate = duet.fluence_to_rate(band1_fluence)
req_exp = calc_exposure(siglimit, band1_rate, bgd_band1, duet.read_noise, duet.neff)
print('High-Zodi {}-sig, {} seconds'.format(siglimit, req_exp.value))

# High Zodi, 5-sigma case
siglimit=5
[bgd_band1, bgd_band2] = background_pixel_rate(duet, high_zodi=True)
band1_fluence, foo = bb_abmag_fluence(duet = duet, swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K)
band1_rate = duet.fluence_to_rate(band1_fluence)

req_exp = calc_exposure(siglimit, band1_rate, bgd_band1, duet.read_noise, duet.neff)
print('High-Zodi {}-sig, {} seconds'.format(siglimit, req_exp.value))



In [11]:
# FOM: Time to 22 ABmag for this configuration for DUET1
# Old FOM for this 
duet = config.Telescope(config='minimum_mass')
duet.read_noise = 3


bandone = duet.bandpass1
bandtwo = duet.bandpass2

siglimit=10
swiftmag = 20
dmag = 0.05
bbmag1 = 5*u.ABmag
bbmag_target = 22*u.ABmag
while bbmag1 < bbmag_target:
    swiftmag+=dmag
    bbmag1, bbmag2 = bb_abmag(swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, bandone = bandone, bandtwo=bandtwo)


# Low Zodi, 10-sigma case
[bgd_band1, bgd_band2] = background_pixel_rate(duet, low_zodi=True)
band1_fluence, foo = bb_abmag_fluence(duet = duet, swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K)
band1_rate = duet.fluence_to_rate(band1_fluence)
req_exp = calc_exposure(siglimit, band1_rate, bgd_band1, duet.read_noise, duet.neff)
print('Low-Zodi {}-sig, {} seconds'.format(siglimit, req_exp.value))


# High Zodi, 10-sigma case.
siglimit=10
[bgd_band1, bgd_band2] = background_pixel_rate(duet, high_zodi=True)
band1_fluence, foo = bb_abmag_fluence(duet = duet, swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K)
band1_rate = duet.fluence_to_rate(band1_fluence)
req_exp = calc_exposure(siglimit, band1_rate, bgd_band1, duet.read_noise, duet.neff)
print('High-Zodi {}-sig, {} seconds'.format(siglimit, req_exp.value))

# High Zodi, 5-sigma case
siglimit=5
[bgd_band1, bgd_band2] = background_pixel_rate(duet, high_zodi=True)
band1_fluence, foo = bb_abmag_fluence(duet = duet, swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K)
band1_rate = duet.fluence_to_rate(band1_fluence)

req_exp = calc_exposure(siglimit, band1_rate, bgd_band1, duet.read_noise, duet.neff)
print('High-Zodi {}-sig, {} seconds'.format(siglimit, req_exp.value))




Low-Zodi 10-sig, 1110.4181967543602 seconds
High-Zodi 10-sig, 1824.2163162763466 seconds
High-Zodi 5-sig, 526.2848228170415 seconds
